# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting info from tensors
* Manipulating Tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try

## Introduction to Tensors


In [ ]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

In [ ]:
# Check the dimension of our vector
vector.ndim

In [ ]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

In [ ]:
matrix.ndim

In [ ]:
# Create a matrix with a different datatype
another_matrix = tf.constant([[10., 7.], [3., 2.], [8., 9.]], dtype=tf.float16)
another_matrix


In [ ]:
# Check # of dims in another_matrix
another_matrix.ndim

In [ ]:
# Create a tensor
tensor = tf.constant([[
                       [1, 2, 3,],
                       [4, 5, 6,]
                      ],[
                        [7, 8, 9], 
                        [10, 11, 12],
                      ],[
                        [13, 14, 15], 
                        [16, 17, 18]
                       ]
                      ])
tensor

In [ ]:
# Check # of dims in tensor
tensor.ndim

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [ ]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

In [ ]:
# Changing one of the elements in the changeable tensor
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Try changing one of the elements in the unchangeable tensor
# unchangeable_tensor[0].assign(7) # (errors out)
unchangeable_tensor

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers.

Random tensors are used when initializing weights for machine learning

In [ ]:
# Create 2 random tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility 
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1 == random_2

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[1,2], [3,4], [5,6], [7,8], [9,10]])
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled)
not_shuffled, shuffled

🔧 **Exercise:** Read through TensorFlow docs on random seed generation: [random/set_seed](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)

Practice writing 5 random tensors and shuffle them.

In [ ]:
# Create 2 random tensors
r = tf.random.Generator.from_seed(42) # set seed for reproducibility 
r1 = r.normal(shape=(4, 2))
r2 = r.normal(shape=(5, 2))
r1, r2

In [ ]:
# Shuffle (unset global seed)
r1_shuffle = tf.random.shuffle(r1) # normal shuffle with no seed (default global is auto set), should be different between runs
# Set operational seed
r1_local = tf.random.shuffle(r1, seed=1) # shuffle with only operational seed, (default global is auto set, then additionally uses operational seed)
r1_shuffle, r1_local

In [ ]:
import random
tf.random.set_seed(random.randint(1, 1000)) # setting seed randomly to emulate an empty seed since the 2nd time it's set (below) keeps the set between runs in this notebook
r2_shuffle = tf.random.shuffle(r2) # Unseeded, randomized every time it's run
# Set global seed
tf.random.set_seed(42)
r2_global = tf.random.shuffle(r2) # No operational seed results in the same shuffle every run using only globally set seed
# Set operational seed
r2_local_global = tf.random.shuffle(r2, seed=1) # Uses both operational and global seed to shuffle, results in the same shuffle every run
r2_local_global_2 = tf.random.shuffle(r2, seed=2) # Using a different operational seed creates a different shuffle than 2_local_global but still remains the same between runs
r2_shuffle, r2_global, r2_local_global, r2_local_global_2

In [ ]:
# Create a tensor of all 1s
tf.ones([10, 7])

In [ ]:
# Create a tensor of all 0s
tf.zeros(shape=(3,4))

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU

X = tf.constant(some_matrix) # Capitals are often associated with matricies or tensors

y = tf.constant(vector) # While lowercase are associated with vectors

In [ ]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # Creates a NumPy array between 1 and 25
numpy_A

In [ ]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

In [ ]:
3 * 8

In [ ]:
A.ndim

### Getting information from tensors

* **Shape**: Length (number of elements) of each of the dimensions of a tensor.
  * `tensor.shape`
* **Rank**: Number of dimensions of a tensor. 
  * Scaler = rank 0
  * Vector = rank 1
  * Matrix = rank 2 
  * Tensor = rank n
  * `tensor.ndim`
* **Axis or Dimension**: A particular dimension of a tensor.
  * `tensor[0], tensor[:, 1]...`
* **Size**: The total number of items in the tensor.
  * `tf.size(tensor)`

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor), rank_4_tensor[0]

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

### Indexing tensors

Tensors can be indexed just like Python lists.

In [ ]:
some_list = [1, 2, 3, 4]
some_list[:2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

In [ ]:
some_list[:1] # gets the first element (0-1)

In [ ]:
# Get the first dimension from each index except for the final one
rank_4_tensor[:1,:1, :1] # or rank_4_tensor[:1,:1, :1, :]

In [ ]:
rank_4_tensor[:1,:1, :, :1]

In [ ]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[1,2], [3,4]])  # alternatively => tf.zeros(shape=[2, 2])
rank_2_tensor.shape, rank_2_tensor.ndim

In [ ]:
# Get the last item of each row of the rank_2_tensor
rank_2_tensor[:, -1]

In [ ]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand the final axis

### Manipulating tensors (tensor operations)

***Basic operations***

`+`, `-`, `*`, `/`


In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[1, 2], [3, 4]])
tensor + 10

In [ ]:
# Original tensor is unchanged
tensor

In [ ]:
# Multiplication also works
tensor * 10

In [ ]:
# We can use the tensorflow built in functions
tf.multiply(tensor, 10) # using tf functions is usually faster

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are 2 rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the inner dimensions

In [ ]:
tf.matmul(tensor, tensor)

In [ ]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

In [ ]:
X = tf.constant([[1, 2, 3], [4, 5, 6]])
Y = tf.constant([[1, 2], [3, 4], [5, 6]])
tf.matmul(X, Y)

In [ ]:
tf.matmul(Y, X)

In [ ]:
r1 = tf.reshape(Y, shape=[3, 2])
r2 = tf.reshape(Y, shape=[2, 3])
r1, r2

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(3, 2)))

In [ ]:
# Can do the same with transpose except transpose also flips the axises
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

In [ ]:
# Try matrix multiplication
A = tf.constant([[1, 2], [3, 4], [5, 6]])
B = tf.constant([[1, 2], [3, 4], [5, 6]])
tf.matmul(A, tf.reshape(B, shape=(2, 3)))

In [ ]:
# With transpose rather than reshape
tf.matmul(A, tf.transpose(B))
# results in different dot product due to axis flip

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()` 


In [ ]:
# Transposed
transposed = tf.transpose(B)
print(tf.matmul(A, transposed))
print(tf.tensordot(A, transposed, axes=1))

In [ ]:
# Reshaped
reshaped = tf.reshape(B, shape=(2, 3))
print(tf.matmul(A, reshaped))
print(tf.tensordot(A, reshaped, axes=1))

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplication rules.

### Changing the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
C = tf.constant([7, 10])
print(B.dtype, C.dtype)

In [ ]:
# Change from float32 to float16
D = tf.cast(B, dtype=tf.float16)
D.dtype


In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E.dtype

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16.dtype

### Aggregating Tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [ ]:
# Get the absolute values with `tf.abs()`
F = tf.constant(np.random.randint(-50, 50, 50))
Absolutes = tf.abs(F).numpy()
# Min
Min = tf.reduce_min(F).numpy()
# Max
Max = tf.reduce_max(F).numpy()
# Mean
Mean = tf.reduce_mean(F).numpy()
# Sum
Sum = tf.reduce_sum(F).numpy()
print("Original Tensor:\n%s\n\nAbsolute Values:\n%s\n\nMin: %s\n\nMax: %s\n\nMean: %s\n\nSum: %s"%(F.numpy(), Absolutes, Min, Max, Mean, Sum))

🔧 **Exercise:** With what we've just learned, find the variance and standard deviation of our `F` tensor using TensorFlow methods.

In [ ]:
import tensorflow_probability as tfp
#Variance
cast = tf.cast(F, dtype=tf.float32)
Variance = tf.math.reduce_variance(cast).numpy()

#STD
Std = tf.math.reduce_std(cast).numpy()
print("Variance: %s\n\nStandard Deviation: %s\n"%(Variance, Std))

# Alternatively, use tfp.stats.variance for rounded int
VarianceTfp = tfp.stats.variance(F).numpy()
print("TFP Variance: %s"%VarianceTfp)

### Find the positional minimum and maximum

In [ ]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

In [ ]:
# Find the positional maximum (argmax)
tf.argmax(F)

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

In [ ]:
# Find the max value of F
tf.reduce_max(F)

In [ ]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
# Find the positional minimum
tf.argmin(F)

In [ ]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

In [ ]:
# Find the minimum using reduce_min
tf.reduce_min(F)

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

In [ ]:
G.shape

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed.shape

In [ ]:
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G_squeezed = tf.squeeze(G)
print(G.shape, G_squeezed.shape)

### One-hot encoding tensors

In [ ]:
# Create a list of indicies
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indicies
tf.one_hot(some_list, depth=4)

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value='true', off_value='false')

### Squaring, Log, Square Root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)

# Square it
squared = tf.square(H)

# SquareRoot (method requires non-int type)
float32Range = tf.cast(H, dtype=tf.float32)
sqrt = tf.sqrt(float32Range)

# Log (method requires non-int type)
log = tf.math.log(float32Range)
print("squared:", squared.numpy(), '\n')
print("sqrt:", sqrt.numpy(), '\n')
print("log:", log.numpy(), '\n')

### Tensors and Numpy

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

In [ ]:
# Convert the tensor back to a NumPy array
np.array(J), type(np.array(J))

In [ ]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

In [ ]:
J = tf.constant([3.])
J.numpy()[0]

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

### Finding access to GPUs

In [ ]:
tf.config.list_physical_devices()